<a href="https://colab.research.google.com/github/lucasl02/SyncodeMath/blob/main/syncode_mathematical_programming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Syncode Inference

In [ ]:
!pip install syncode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.1/206.1 kB 17.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 796.9/796.9 kB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 100.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 123.2 MB/s eta 0:00:00
  Created wheel for syncode: filename=syncode-0.4.16-py3-none-any.whl size=237008 sha256=18255dcfc5960c665ebefa8cb68cd81b39b4c89e59ea1a7a1021119f80df7d0a
  Stored in directory: /root/.cache/pip/wheels/3f/a5/9b/9f6e1723cd27ffd3feea8185b6d3cd8578d22e8e240dc3ca9d
Successfully built syncode
  Attempting uni

In [ ]:
from syncode import Syncode
import warnings
warnings.filterwarnings('ignore')

model_name = "microsoft/phi-2"

# Load the Syncode augmented model
syn_llm = Syncode(model=model_name, grammar='json', max_new_tokens=400)

In [ ]:
import json
import operator

def run_syncode(textual_query):
    """
    Takes a textual query, uses Syncode to parse operands and operator,
    and returns (operands, operator, final_answer).
    """
    # 1. mnake prompt
    # use example as a one shot encoding exmaple
    prompt = f"""Parse the arithmetic query into a JSON object with keys "operands" (list of numbers) and "operator" (string: "+", "-", "*", "/").

              EXAMPLES:
              Query: What is 327. multiplied by 11.0?
              JSON: {{ "operands": [327.0, 11.0], "operator": "*" }}

              Query: What is 45.1 plus 23.54?
              JSON: {{ "operands": [45.1, 23.54], "operator": "+" }}

              NOW:
              Query: {textual_query}
              JSON: """

    output = syn_llm.infer(prompt)[0]
    # raw output has lots of empty lines
    #print(f"Raw output: {output}")

    # get rid of all the white space
    start_idx = output.find('{')
    end_idx = output.rfind('}')

    if start_idx != -1 and end_idx != -1:
        json_str = output[start_idx : end_idx + 1]
        data = json.loads(json_str)
    else:
        data = json.loads(output)

    # get operands
    operands = data.get("operands", [])
    operands = [float(x) for x in operands]


    # get operator
    op_symbol = data.get("operator", "")


    answer = 0.0
    # make sure we have two number sbeing returned
    if len(operands) >= 2:
        if op_symbol == '+':
            answer = operands[0] + operands[1]
        elif op_symbol == '-':
            answer = operands[0] - operands[1]
        elif op_symbol == '*':
            answer = operands[0] * operands[1]
        elif op_symbol == '/':
            answer = operands[0] / operands[1]
    #print((operands, op_symbol, float(answer)))
    return (operands, op_symbol, float(answer))

In [ ]:
queries = ["What is 327. multiplied by 11.0?",
    "What is 45.1 plus 23.54?",
    "What is 120.4 divided by 4.0?"]

for q in queries:
    answer = run_syncode(q)
    print(f"Query: {q}")
    print(f"Result: {answer}")
    print()

Query: What is 327. multiplied by 11.0?
Result: 3597.0

Query: What is 45.1 plus 23.54?
Result: 68.64

Query: What is 120.4 divided by 4.0?
Result: 30.1

